1. Our models are underperforming (not fitting the data well). What are 3 methods for preventing underfitting? Write them down and explain each with a sentence.

* If the model is too simple for the dataset, then it will underfit.
* Making sure we have enough training data.
* Regularisation: It is the process of penalising the loss function, which discourages the  model from using overly complex solutions.

2. Recreate the data loading functions we built in sections 1, 2, 3 and 4. You should have train and test DataLoader's ready to use.

In [1]:
from pathlib import Path 

DATA_PATH= Path('../data')
IMAGE_PATH= DATA_PATH/'pizza_steak_sushi_20_percent'

train_dir= IMAGE_PATH/'train'
test_dir= IMAGE_PATH/'test'

In [2]:
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms
import os

############### TRANSFORM FUNCTION ###############
train_transform= transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transform= transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

############### DATASETS ###############
train_data= datasets.ImageFolder(root=train_dir,
                                 transform=train_transform,
                                 target_transform=None)

test_data= datasets.ImageFolder(root=test_dir,
                                transform=test_transform,
                                target_transform=None)

BATCH_SIZE= 32
NUM_WORKER= os.cpu_count()

############### DATALOADERS ###############
train_dataloader= DataLoader(dataset= train_data,
                             batch_size= BATCH_SIZE,
                             shuffle= True,
                             num_workers= NUM_WORKER)

test_dataloader= DataLoader(dataset= test_data,
                            batch_size= 32,
                            shuffle= False,
                            num_workers= NUM_WORKER)

In [3]:
image_batch, label_batch = next(iter(train_dataloader))

In [4]:
class_names= train_data.classes
len(class_names)
print(image_batch)


tensor([[[[0.0863, 0.0902, 0.0863,  ..., 0.0588, 0.0588, 0.0627],
          [0.0863, 0.0863, 0.0902,  ..., 0.0784, 0.0706, 0.0706],
          [0.0863, 0.0824, 0.0824,  ..., 0.2863, 0.1098, 0.0784],
          ...,
          [0.7725, 0.7686, 0.7647,  ..., 0.8588, 0.8471, 0.8118],
          [0.5961, 0.6627, 0.7137,  ..., 0.6706, 0.5882, 0.5294],
          [0.2235, 0.2902, 0.3882,  ..., 0.4039, 0.3765, 0.3882]],

         [[0.0863, 0.0902, 0.0863,  ..., 0.0549, 0.0510, 0.0549],
          [0.0863, 0.0863, 0.0902,  ..., 0.0784, 0.0627, 0.0627],
          [0.0863, 0.0824, 0.0824,  ..., 0.2863, 0.0980, 0.0667],
          ...,
          [0.2314, 0.2235, 0.2235,  ..., 0.3608, 0.4039, 0.4471],
          [0.2039, 0.2118, 0.2196,  ..., 0.4549, 0.4118, 0.3725],
          [0.1059, 0.1137, 0.1333,  ..., 0.2863, 0.2627, 0.2627]],

         [[0.1255, 0.1216, 0.1176,  ..., 0.0706, 0.0627, 0.0667],
          [0.1255, 0.1176, 0.1216,  ..., 0.0863, 0.0745, 0.0745],
          [0.1255, 0.1137, 0.1137,  ..., 0

3. Recreate `model_0` we built in section 7.

In [5]:
from torch import nn

class TinyVGG(nn.Module):
    def __init__(self,
                 input_shape: int,
                 output_shape: int,
                 hidden_units: int= 10) -> None:
        super().__init__()
        self.covo_block_1= nn.Sequential(
            nn.Conv2d(in_channels= input_shape,
                      out_channels= hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels= hidden_units,
                      out_channels= hidden_units,
                      kernel_size= 3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.covo_block_2= nn.Sequential(
            nn.Conv2d(in_channels= hidden_units,
                      out_channels= hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels= hidden_units,
                      out_channels= hidden_units,
                      kernel_size= 3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier= nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= hidden_units*13*13,
                      out_features= output_shape)
        )

    def forward(self, x):
        x= self.covo_block_1(x)
        # print(x)
        x= self.covo_block_2(x)
        # print(x)
        x= self.classifier(x)
        return x


In [6]:
from torchinfo import summary
import torch

model_0= TinyVGG(input_shape=3,
                 output_shape=len(class_names),
                 hidden_units=10)

In [7]:
summary(model= model_0, input_size= [1, 3, 64, 64])

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 30, 30]           --
│    └─Conv2d: 2-1                       [1, 10, 62, 62]           280
│    └─ReLU: 2-2                         [1, 10, 62, 62]           --
│    └─Conv2d: 2-3                       [1, 10, 60, 60]           910
│    └─ReLU: 2-4                         [1, 10, 60, 60]           --
│    └─MaxPool2d: 2-5                    [1, 10, 30, 30]           --
├─Sequential: 1-2                        [1, 10, 13, 13]           --
│    └─Conv2d: 2-6                       [1, 10, 28, 28]           910
│    └─ReLU: 2-7                         [1, 10, 28, 28]           --
│    └─Conv2d: 2-8                       [1, 10, 26, 26]           910
│    └─ReLU: 2-9                         [1, 10, 26, 26]           --
│    └─MaxPool2d: 2-10                   [1, 10, 13, 13]           --
├─Sequentia

In [8]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

4. Create training and testing functions for `model_0`

In [9]:
def train_step(model: nn.Module,
               dataloader: DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device= 'cpu'):
    
    # Setting model to training mode
    model.to(device).train()

    # Accumilating train and test loss per batch
    train_loss, train_acc= 0, 0

    for batch, (X,y) in enumerate(dataloader):
        X,y= X.to(device), y.to(device)

        # Forward pass
        y_pred_logit= model(X)

        # Loss caclulation
        loss = loss_fn(y_pred_logit, y)
        train_loss += loss.item()

        # Optimizer Zero Grad
        optimizer.zero_grad()

        # Loss backward
        loss.backward()

        # Optimizer Step
        optimizer.step()

        # Calculating Accuracy
        y_pred_label= torch.argmax(torch.softmax(y_pred_logit, dim=1), dim=1)
        train_acc += (y_pred_label==y).sum().item()/len(y_pred_logit)

    train_loss /= len(dataloader)
    train_acc /= len(dataloader)
    return train_loss, train_acc

In [10]:
def test_step(model: nn.Module,
              dataloader: DataLoader,
              loss_fn: nn.Module,
              device: torch.device= 'cpu'):
    
    # Setting model to evaluation mode
    model.to(device).eval()

    # Accumilating test loss and acc per batch
    test_loss, test_acc= 0, 0

    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            # Setting all variables to same device
            X, y= X.to(device), y.to(device)

            # Forward Pass
            test_pred_logits= model(X)
            
            # Loss Calculation
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Accuracy Calculation
            test_pred_labels= torch.argmax(torch.softmax(test_pred_logits, dim=1), dim=1)
            test_acc+= (test_pred_labels==y).sum().item()/len(test_pred_labels)

        test_loss /= len(dataloader)
        test_acc /= len(dataloader)
    return test_loss, test_acc

5. Try training the model you made in exercise 3 for 5, 20 and 50 epochs, what happens to the results?
    * Use `torch.optim.Adam()` with a learning rate of 0.001 as the optimizer.

In [11]:
from tqdm import tqdm
def train(model: nn.Module,
          train_dataloader: DataLoader,
          test_dataloader: DataLoader,
          epochs: int,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          device: torch.device= 'cpu'):
    
    results={'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []}
    
    for epoch in tqdm(range(epochs)):

        # Training
        train_loss, train_acc= train_step(model= model,
                                          dataloader= train_dataloader,
                                          loss_fn= loss_fn,
                                          optimizer= optimizer,
                                          device= device)

        # Testing
        test_loss, test_acc= test_step(model=model,
                                       dataloader= test_dataloader,
                                       loss_fn= loss_fn,
                                       device= device)

        print(f"Epoch: {epoch} | Train Loss: {train_loss:.3f} | Train Accuracy: {100*train_acc:.2f}% | Test Loss: {test_loss:.3f} | Test Accuracy: {100*test_acc:.2f}%")
        
        # Storing results
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)
        
    return results      

In [12]:
EPOCHS= 5
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(params= model_0.parameters(), lr= 0.001)

model_0_results_5_epochs= train(model= model_0,
                                train_dataloader= train_dataloader,
                                test_dataloader= test_dataloader,
                                epochs= EPOCHS,
                                loss_fn= loss_fn,
                                optimizer= optimizer,
                                device= device)

 20%|██        | 1/5 [00:17<01:08, 17.16s/it]

Epoch: 0 | Train Loss: 1.100 | Train Accuracy: 36.04% | Test Loss: 1.082 | Test Accuracy: 48.35%


 40%|████      | 2/5 [00:36<00:55, 18.57s/it]

Epoch: 1 | Train Loss: 1.071 | Train Accuracy: 42.92% | Test Loss: 1.025 | Test Accuracy: 47.95%


 60%|██████    | 3/5 [00:54<00:36, 18.30s/it]

Epoch: 2 | Train Loss: 0.991 | Train Accuracy: 51.25% | Test Loss: 0.961 | Test Accuracy: 52.73%


 80%|████████  | 4/5 [01:09<00:16, 16.83s/it]

Epoch: 3 | Train Loss: 1.019 | Train Accuracy: 50.00% | Test Loss: 0.999 | Test Accuracy: 52.39%


100%|██████████| 5/5 [01:26<00:00, 17.29s/it]

Epoch: 4 | Train Loss: 1.045 | Train Accuracy: 43.96% | Test Loss: 0.971 | Test Accuracy: 52.67%


In [13]:
EPOCHS= 20
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(params= model_0.parameters(), lr= 0.001)

model_0_results_20_epochs= train(model= model_0,
                                train_dataloader= train_dataloader,
                                test_dataloader= test_dataloader,
                                epochs= EPOCHS,
                                loss_fn= loss_fn,
                                optimizer= optimizer,
                                device= device)

  5%|▌         | 1/20 [00:16<05:18, 16.78s/it]

Epoch: 0 | Train Loss: 0.953 | Train Accuracy: 48.96% | Test Loss: 0.932 | Test Accuracy: 52.78%


 10%|█         | 2/20 [00:31<04:41, 15.66s/it]

Epoch: 1 | Train Loss: 0.886 | Train Accuracy: 56.04% | Test Loss: 0.908 | Test Accuracy: 57.33%


 15%|█▌        | 3/20 [00:47<04:30, 15.92s/it]

...

Epoch: 17 | Train Loss: 0.815 | Train Accuracy: 61.67% | Test Loss: 0.878 | Test Accuracy: 58.13%


 95%|█████████▌| 19/20 [04:31<00:13, 13.91s/it]

Epoch: 18 | Train Loss: 0.709 | Train Accuracy: 71.04% | Test Loss: 0.859 | Test Accuracy: 55.34%


100%|██████████| 20/20 [04:46<00:00, 14.33s/it]

Epoch: 19 | Train Loss: 0.722 | Train Accuracy: 65.42% | Test Loss: 0.858 | Test Accuracy: 58.47%


In [14]:
EPOCHS= 50
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(params= model_0.parameters(), lr= 0.001)

model_0_results_50_epochs= train(model= model_0,
                                train_dataloader= train_dataloader,
                                test_dataloader= test_dataloader,
                                epochs= EPOCHS,
                                loss_fn= loss_fn,
                                optimizer= optimizer,
                                device= device)

  2%|▏         | 1/50 [00:14<11:36, 14.22s/it]

Epoch: 0 | Train Loss: 0.768 | Train Accuracy: 66.04% | Test Loss: 0.869 | Test Accuracy: 60.40%


  4%|▍         | 2/50 [00:30<12:12, 15.26s/it]

Epoch: 1 | Train Loss: 0.676 | Train Accuracy: 67.29% | Test Loss: 0.888 | Test Accuracy: 59.38%


  6%|▌         | 3/50 [00:43<11:11, 14.29s/it]

...

Epoch: 47 | Train Loss: 0.276 | Train Accuracy: 89.17% | Test Loss: 1.619 | Test Accuracy: 52.56%


 98%|█████████▊| 49/50 [10:48<00:13, 13.10s/it]

Epoch: 48 | Train Loss: 0.183 | Train Accuracy: 92.29% | Test Loss: 1.739 | Test Accuracy: 53.52%


100%|██████████| 50/50 [11:03<00:00, 13.27s/it]

Epoch: 49 | Train Loss: 0.153 | Train Accuracy: 95.83% | Test Loss: 1.937 | Test Accuracy: 55.34%


6. Double the number of hidden units in your model and train it for 20 epochs, what happens to the results?

In [15]:
model_1= TinyVGG(input_shape=3,
                 output_shape=len(class_names),
                 hidden_units=20)

In [16]:
EPOCHS= 20
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(params= model_1.parameters(), lr= 0.001)

model_1_results= train(model= model_1, 
                       train_dataloader= train_dataloader,
                       test_dataloader= test_dataloader,
                       epochs= EPOCHS,
                       loss_fn= loss_fn,
                       optimizer= optimizer,
                       device= device)

  5%|▌         | 1/20 [00:13<04:12, 13.28s/it]

Epoch: 0 | Train Loss: 1.099 | Train Accuracy: 31.46% | Test Loss: 1.096 | Test Accuracy: 34.72%


 10%|█         | 2/20 [00:27<04:14, 14.12s/it]

Epoch: 1 | Train Loss: 1.092 | Train Accuracy: 31.67% | Test Loss: 1.068 | Test Accuracy: 41.59%


 15%|█▌        | 3/20 [00:40<03:49, 13.52s/it]

...

Epoch: 17 | Train Loss: 0.651 | Train Accuracy: 72.92% | Test Loss: 0.961 | Test Accuracy: 59.77%


 95%|█████████▌| 19/20 [04:13<00:12, 12.52s/it]

Epoch: 18 | Train Loss: 0.673 | Train Accuracy: 67.92% | Test Loss: 0.906 | Test Accuracy: 57.50%


100%|██████████| 20/20 [04:26<00:00, 13.35s/it]

Epoch: 19 | Train Loss: 0.625 | Train Accuracy: 73.96% | Test Loss: 0.956 | Test Accuracy: 58.47%


7. Double the data you're using with your model and train it for 20 epochs, what happens to the results?
    * **Note**: You can use the custom data creation notebook to scale up your Food101 dataset.
    * You can also find the already formatted double data (20% instead of 10% subset) dataset on GitHub, you will need to write download code like in exercise 2 to get it into this notebook.

### Downloading More data from Food101 Dataset

In [17]:
# Setup data directory
import pathlib
data_dir = pathlib.Path("../data")

In [18]:
# Get training data
train_data = datasets.Food101(root=data_dir,
                              split="train",
                              # transform=transforms.ToTensor(),
                              download=True)

# Get testing data
test_data = datasets.Food101(root=data_dir,
                             split="test",
                             # transform=transforms.ToTensor(),
                             download=True)

In [19]:
train_data

Dataset Food101
    Number of datapoints: 75750
    Root location: ..\data
    split=train

In [20]:
class_names = train_data.classes
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [21]:
import random 
import pathlib

# Setup data paths
data_path = DATA_PATH / 'food-101'/ 'images'
target_classes= ['pizza', 'steak', 'sushi']

# Change amount to get (0.2 = random 20%)
amount_to_get= 0.2

def get_subset(image_path= data_path,
               data_splits= ['train', 'test'],
               target_classes=['pizza', 'steak', 'sushi'],
               amount= amount_to_get,
               seed= 42):
    
    random.seed(seed)
    label_splits= {}

    # Get labels 
    for data_split in data_splits:
        print(f"[INFO] Creating image split for {data_split}...")
        label_path= DATA_PATH/ 'food-101'/'meta'/f'{data_split}.txt'
        with open(label_path, 'r') as f:
            labels = [line.strip("\n") for line in f.readlines() if line.split("/")[0] in target_classes] 
        
        # Get random subset of target classes image ID's
        number_to_sample = round(amount * len(labels))
        print(f"[INFO] Getting random subset of {number_to_sample} images for {data_split}...")
        sampled_images = random.sample(labels, k=number_to_sample)
        
        # Apply full paths
        image_paths = [pathlib.Path(str(image_path / sample_image) + ".jpg") for sample_image in sampled_images]
        label_splits[data_split] = image_paths
    return label_splits
        
label_splits = get_subset(amount=amount_to_get)
label_splits["train"][:10]

[INFO] Creating image split for train...
[INFO] Getting random subset of 450 images for train...
[INFO] Creating image split for test...
[INFO] Getting random subset of 150 images for test...


[WindowsPath('../data/food-101/images/pizza/3269634.jpg'),
 WindowsPath('../data/food-101/images/pizza/1524655.jpg'),
 WindowsPath('../data/food-101/images/steak/2825100.jpg'),
 WindowsPath('../data/food-101/images/steak/225990.jpg'),
 WindowsPath('../data/food-101/images/steak/1839481.jpg'),
 WindowsPath('../data/food-101/images/pizza/38349.jpg'),
 WindowsPath('../data/food-101/images/pizza/3018077.jpg'),
 WindowsPath('../data/food-101/images/sushi/93139.jpg'),
 WindowsPath('../data/food-101/images/pizza/2702825.jpg'),
 WindowsPath('../data/food-101/images/sushi/200025.jpg')]

In [22]:
# Create target directory path
target_dir_name = f"../data/pizza_steak_sushi_{str(int(amount_to_get*100))}_percent"
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories
target_dir = pathlib.Path(target_dir_name)

# Make the directories
target_dir.mkdir(parents=True, exist_ok=True)

Creating directory: '../data/pizza_steak_sushi_20_percent'


In [23]:
import shutil

for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        print(f"[INFO] Copying {image_path} to {dest_dir}...")
        shutil.copy2(image_path, dest_dir)

[INFO] Copying ..\data\food-101\images\pizza\3269634.jpg to ..\data\pizza_steak_sushi_20_percent\train\pizza\3269634.jpg...
[INFO] Copying ..\data\food-101\images\pizza\1524655.jpg to ..\data\pizza_steak_sushi_20_percent\train\pizza\1524655.jpg...
[INFO] Copying ..\data\food-101\images\steak\2825100.jpg to ..\data\pizza_steak_sushi_20_percent\train\steak\2825100.jpg...
[INFO] Copying ..\data\food-101\images\steak\225990.jpg to ..\data\pizza_steak_sushi_20_percent\train\steak\225990.jpg...
[INFO] Copying ..\data\food-101\images\steak\1839481.jpg to ..\data\pizza_steak_sushi_20_percent\train\steak\1839481.jpg...
[INFO] Copying ..\data\food-101\images\pizza\38349.jpg to ..\data\pizza_steak_sushi_20_percent\train\pizza\38349.jpg...
[INFO] Copying ..\data\food-101\images\pizza\3018077.jpg to ..\data\pizza_steak_sushi_20_percent\train\pizza\3018077.jpg...
[INFO] Copying ..\data\food-101\images\sushi\93139.jpg to ..\data\pizza_steak_sushi_20_percent\train\sushi\93139.jpg...
[INFO] Copying ..\

In [24]:
# Check lengths of directories
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  import os
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    
walk_through_dir(target_dir)

There are 2 directories and 0 images in '..\data\pizza_steak_sushi_20_percent'.
There are 3 directories and 0 images in '..\data\pizza_steak_sushi_20_percent\test'.
There are 0 directories and 46 images in '..\data\pizza_steak_sushi_20_percent\test\pizza'.
There are 0 directories and 58 images in '..\data\pizza_steak_sushi_20_percent\test\steak'.
There are 0 directories and 46 images in '..\data\pizza_steak_sushi_20_percent\test\sushi'.
There are 3 directories and 0 images in '..\data\pizza_steak_sushi_20_percent\train'.
There are 0 directories and 154 images in '..\data\pizza_steak_sushi_20_percent\train\pizza'.
There are 0 directories and 146 images in '..\data\pizza_steak_sushi_20_percent\train\steak'.
There are 0 directories and 150 images in '..\data\pizza_steak_sushi_20_percent\train\sushi'.


In [25]:
# Zip pizza_steak_sushi images
zip_file_name = data_dir / f"pizza_steak_sushi_{str(int(amount_to_get*100))}_percent"
shutil.make_archive(zip_file_name, 
                    format="zip", 
                    root_dir=target_dir)

'c:\\Users\\Kushagra\\Desktop\\PyTorch_Course\\data\\pizza_steak_sushi_20_percent.zip'

In [26]:
walk_through_dir("../data/pizza_steak_sushi_20_percent")

There are 2 directories and 0 images in '../data/pizza_steak_sushi_20_percent'.
There are 3 directories and 0 images in '../data/pizza_steak_sushi_20_percent\test'.
There are 0 directories and 46 images in '../data/pizza_steak_sushi_20_percent\test\pizza'.
There are 0 directories and 58 images in '../data/pizza_steak_sushi_20_percent\test\steak'.
There are 0 directories and 46 images in '../data/pizza_steak_sushi_20_percent\test\sushi'.
There are 3 directories and 0 images in '../data/pizza_steak_sushi_20_percent\train'.
There are 0 directories and 154 images in '../data/pizza_steak_sushi_20_percent\train\pizza'.
There are 0 directories and 146 images in '../data/pizza_steak_sushi_20_percent\train\steak'.
There are 0 directories and 150 images in '../data/pizza_steak_sushi_20_percent\train\sushi'.


### Updated directory paths above and re-ran trained the model on new data

8. Make a prediction on your own custom image of pizza/steak/sushi (you could even download one from the internet) and share your prediction.
* Does the model you trained in exercise 7 get it right?
* If not, what do you think you could do to improve it?

* The model improved by almost 20 percent, to improve the model we can do the following things to get better accuracy:
    1. Get more training data
    2. Add hidden units
    3. Apply Reinforcement learning on the loss function
    4. train for more epochs (**not recommended since it may lead to overfitting after a while**)